<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/SensorAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from collections import deque
from typing import Dict, List, Optional, Tuple, Union
import warnings
warnings.filterwarnings('ignore')

# Core ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats
from scipy.spatial.distance import jensenshannon

# Deep learning (choose one: keras or torch)
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
    from tensorflow.keras.optimizers import Adam
    KERAS_AVAILABLE = True
except ImportError:
    KERAS_AVAILABLE = False

# Statistical modeling
try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    STATSMODELS_AVAILABLE = True
except ImportError:
    STATSMODELS_AVAILABLE = False


class SensorAgent:
    """
    Autonomous Sensor Agent for IoT time series anomaly and drift detection.

    This agent operates independently on a single sensor's data stream,
    maintaining its own model, memory, and adaptive thresholding mechanisms.
    Designed for integration into larger multi-agent frameworks.
    """

    def __init__(self,
                 sensor_id: str,
                 model_type: str = 'lstm_autoencoder',  # 'lstm_autoencoder', 'vae', 'arima'
                 window_length: int = 50,
                 memory_size: int = 1000,
                 threshold_k: float = 2.0,
                 drift_threshold: float = 0.1,
                 min_training_samples: int = 100):
        """
        Initialize Sensor Agent.

        Args:
            sensor_id: Unique identifier for this sensor
            model_type: Type of model ('lstm_autoencoder', 'vae', 'arima')
            window_length: Length of input subsequences
            memory_size: Size of rolling memory buffer
            threshold_k: Multiplier for adaptive threshold (mean + k*std)
            drift_threshold: Threshold for drift detection
            min_training_samples: Minimum samples needed before training
        """
        self.sensor_id = sensor_id
        self.model_type = model_type
        self.window_length = window_length
        self.memory_size = memory_size
        self.threshold_k = threshold_k
        self.drift_threshold = drift_threshold
        self.min_training_samples = min_training_samples

        # Memory buffers - core of agentic behavior
        self.error_memory = deque(maxlen=memory_size)
        self.data_memory = deque(maxlen=memory_size)
        self.recent_errors = deque(maxlen=100)  # For drift detection

        # Model and preprocessing
        self.model = None
        self.scaler = StandardScaler()
        self.is_fitted = False

        # Adaptive statistics
        self.rolling_stats = {
            'mean': 0.0,
            'std': 1.0,
            'q95': 0.0,
            'q99': 0.0
        }

        # Drift detection state
        self.baseline_errors = None
        self.drift_detected_count = 0

        # Agent state
        self.total_processed = 0
        self.anomalies_detected = 0
        self.last_retrain_time = None

    def _build_lstm_autoencoder(self) -> Model:
        """Build LSTM Autoencoder model."""
        if not KERAS_AVAILABLE:
            raise ImportError("Keras/TensorFlow not available. Install with: pip install tensorflow")

        # Encoder
        inputs = Input(shape=(self.window_length, 1))
        encoded = LSTM(32, activation='relu', return_sequences=False)(inputs)

        # Decoder
        decoded = RepeatVector(self.window_length)(encoded)
        decoded = LSTM(32, activation='relu', return_sequences=True)(decoded)
        outputs = TimeDistributed(Dense(1, activation='linear'))(decoded)

        model = Model(inputs, outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    def _build_vae(self) -> Model:
        """Build Variational Autoencoder model."""
        if not KERAS_AVAILABLE:
            raise ImportError("Keras/TensorFlow not available. Install with: pip install tensorflow")

        latent_dim = 16

        # Encoder
        inputs = Input(shape=(self.window_length, 1))
        x = LSTM(32, return_sequences=False)(inputs)
        z_mean = Dense(latent_dim)(x)
        z_log_var = Dense(latent_dim)(x)

        # Sampling function
        def sampling(args):
            z_mean, z_log_var = args
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon

        z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_var])

        # Decoder
        decoder_input = RepeatVector(self.window_length)(z)
        decoded = LSTM(32, return_sequences=True)(decoder_input)
        outputs = TimeDistributed(Dense(1))(decoded)

        model = Model(inputs, outputs)

        # VAE loss
        reconstruction_loss = tf.reduce_mean(tf.square(inputs - outputs))
        kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        vae_loss = reconstruction_loss + 0.1 * kl_loss
        model.add_loss(vae_loss)
        model.compile(optimizer=Adam(learning_rate=0.001))

        return model

    def fit(self, training_data: np.ndarray) -> None:
        """
        Train the sensor model on historical data.

        Args:
            training_data: Historical sensor data for training
        """
        if len(training_data) < self.min_training_samples:
            print(f"Sensor {self.sensor_id}: Insufficient training data ({len(training_data)} < {self.min_training_samples})")
            return

        print(f"Sensor {self.sensor_id}: Training {self.model_type} model...")

        # Normalize data
        training_data_scaled = self.scaler.fit_transform(training_data.reshape(-1, 1)).flatten()

        # Create sequences for training
        sequences = []
        for i in range(len(training_data_scaled) - self.window_length + 1):
            sequences.append(training_data_scaled[i:i + self.window_length])

        X_train = np.array(sequences)

        if self.model_type == 'lstm_autoencoder':
            self.model = self._build_lstm_autoencoder()
            X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            self.model.fit(X_train_reshaped, X_train_reshaped,
                          epochs=50, batch_size=32, verbose=0, validation_split=0.2)

        elif self.model_type == 'vae':
            self.model = self._build_vae()
            X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            self.model.fit(X_train_reshaped, epochs=50, batch_size=32, verbose=0)

        elif self.model_type == 'arima':
            if not STATSMODELS_AVAILABLE:
                raise ImportError("Statsmodels not available. Install with: pip install statsmodels")
            # For ARIMA, we'll store parameters and fit on-demand
            self.model = 'arima_fitted'  # Placeholder

        # Compute baseline errors for drift detection
        baseline_errors = []
        for seq in X_train:
            error = self._compute_anomaly_score(seq)
            baseline_errors.append(error)

        self.baseline_errors = np.array(baseline_errors)
        self._update_rolling_stats(baseline_errors)

        self.is_fitted = True
        self.last_retrain_time = datetime.now()
        print(f"Sensor {self.sensor_id}: Model training completed")

    def _compute_anomaly_score(self, sequence: np.ndarray) -> float:
        """Compute anomaly score for a sequence."""
        if not self.is_fitted:
            return 0.0

        # Normalize sequence
        sequence_scaled = self.scaler.transform(sequence.reshape(-1, 1)).flatten()

        if self.model_type in ['lstm_autoencoder', 'vae']:
            # Reshape for model input
            X = sequence_scaled.reshape(1, len(sequence_scaled), 1)
            reconstruction = self.model.predict(X, verbose=0)
            reconstruction = reconstruction.reshape(-1)

            # Compute reconstruction error
            error = mean_squared_error(sequence_scaled, reconstruction)

        elif self.model_type == 'arima':
            try:
                # Fit ARIMA model and compute forecast error
                model = ARIMA(sequence_scaled, order=(2, 1, 2))
                fitted_model = model.fit()
                forecast = fitted_model.forecast(steps=1)[0]
                error = abs(sequence_scaled[-1] - forecast)
            except:
                error = 0.0  # Fallback for ARIMA fitting issues

        return float(error)

    def _update_rolling_stats(self, errors: List[float]) -> None:
        """Update rolling statistics for adaptive thresholding."""
        if len(errors) == 0:
            return

        errors_array = np.array(errors)
        self.rolling_stats['mean'] = np.mean(errors_array)
        self.rolling_stats['std'] = np.std(errors_array) + 1e-8  # Avoid division by zero
        self.rolling_stats['q95'] = np.percentile(errors_array, 95)
        self.rolling_stats['q99'] = np.percentile(errors_array, 99)

    def _detect_drift(self) -> Tuple[bool, float]:
        """
        Detect distribution drift in recent errors vs baseline.

        Returns:
            Tuple of (drift_detected, drift_score)
        """
        if self.baseline_errors is None or len(self.recent_errors) < 30:
            return False, 0.0

        recent_errors_array = np.array(list(self.recent_errors))

        # Use Jensen-Shannon divergence for distribution comparison
        try:
            # Create histograms
            hist_baseline, bins = np.histogram(self.baseline_errors, bins=20, density=True)
            hist_recent, _ = np.histogram(recent_errors_array, bins=bins, density=True)

            # Add small epsilon to avoid zero probabilities
            hist_baseline += 1e-10
            hist_recent += 1e-10

            # Normalize to probabilities
            hist_baseline /= hist_baseline.sum()
            hist_recent /= hist_recent.sum()

            # Compute Jensen-Shannon divergence
            js_divergence = jensenshannon(hist_baseline, hist_recent)
            drift_detected = js_divergence > self.drift_threshold

            return drift_detected, float(js_divergence)

        except Exception as e:
            # Fallback to KS test if JS divergence fails
            try:
                ks_stat, p_value = stats.ks_2samp(self.baseline_errors, recent_errors_array)
                drift_detected = p_value < 0.05
                return drift_detected, float(ks_stat)
            except:
                return False, 0.0

    def detect(self, subsequence: np.ndarray, timestamp: datetime = None) -> Dict:
        """
        Main detection method - processes a single subsequence.

        Args:
            subsequence: Input sensor data subsequence (shape: window_length,)
            timestamp: Timestamp for this subsequence

        Returns:
            Detection results dictionary
        """
        if timestamp is None:
            timestamp = datetime.now()

        if len(subsequence) != self.window_length:
            raise ValueError(f"Expected subsequence length {self.window_length}, got {len(subsequence)}")

        # Compute anomaly score
        anomaly_score = self._compute_anomaly_score(subsequence)

        # Update memory (agentic behavior)
        self.data_memory.append(subsequence.copy())
        self.error_memory.append(anomaly_score)
        self.recent_errors.append(anomaly_score)

        # Adaptive thresholding
        current_threshold = self.rolling_stats['mean'] + self.threshold_k * self.rolling_stats['std']
        is_anomaly = anomaly_score > current_threshold

        # Compute confidence based on how far the score is from threshold
        confidence = min(1.0, abs(anomaly_score - current_threshold) / (self.rolling_stats['std'] + 1e-8))

        # Drift detection
        drift_flag, drift_score = self._detect_drift()

        # Update statistics periodically (agentic adaptation)
        if len(self.error_memory) >= 50 and len(self.error_memory) % 10 == 0:
            self._update_rolling_stats(list(self.error_memory)[-50:])

        # Update counters
        self.total_processed += 1
        if is_anomaly:
            self.anomalies_detected += 1

        if drift_flag:
            self.drift_detected_count += 1

        # Prepare output
        result = {
            "sensor_id": self.sensor_id,
            "timestamp": timestamp,
            "anomaly_score": float(anomaly_score),
            "is_anomaly": bool(is_anomaly),
            "drift_flag": bool(drift_flag),
            "confidence": float(confidence),
            # Additional fields for master agent escalation
            "threshold_used": float(current_threshold),
            "drift_score": float(drift_score) if drift_flag else 0.0,
            "total_processed": self.total_processed,
            "anomaly_rate": self.anomalies_detected / max(1, self.total_processed)
        }

        return result

    def update_memory(self, new_data: np.ndarray) -> None:
        """
        Update agent memory with new data batch.
        For continuous learning and adaptation.
        """
        for i in range(len(new_data) - self.window_length + 1):
            subsequence = new_data[i:i + self.window_length]
            self.data_memory.append(subsequence)

    def should_retrain(self) -> bool:
        """
        Determine if model should be retrained based on drift and performance.
        Agentic decision-making capability.
        """
        if self.last_retrain_time is None:
            return False

        # Retrain if significant drift detected or poor recent performance
        time_since_retrain = datetime.now() - self.last_retrain_time
        drift_rate = self.drift_detected_count / max(1, self.total_processed)

        return (time_since_retrain.days > 7 or  # Weekly retraining
                drift_rate > 0.1 or  # High drift rate
                (len(self.error_memory) > 500 and
                 np.mean(list(self.error_memory)[-100:]) > 2 * self.rolling_stats['mean']))

    def get_status(self) -> Dict:
        """Get current agent status and diagnostics."""
        return {
            "sensor_id": self.sensor_id,
            "model_type": self.model_type,
            "is_fitted": self.is_fitted,
            "total_processed": self.total_processed,
            "anomalies_detected": self.anomalies_detected,
            "anomaly_rate": self.anomalies_detected / max(1, self.total_processed),
            "drift_detected_count": self.drift_detected_count,
            "memory_utilization": len(self.error_memory) / self.memory_size,
            "current_threshold": self.rolling_stats['mean'] + self.threshold_k * self.rolling_stats['std'],
            "rolling_stats": self.rolling_stats.copy(),
            "should_retrain": self.should_retrain()
        }


def simulate_sensor_data(num_sensors: int = 12, num_samples: int = 2000,
                        anomaly_prob: float = 0.05) -> Dict[str, np.ndarray]:
    """
    Simulate IoT sensor data for testing.

    Args:
        num_sensors: Number of sensors to simulate
        num_samples: Number of time steps per sensor
        anomaly_prob: Probability of anomaly at each time step

    Returns:
        Dictionary of sensor_id -> time series data
    """
    sensors_data = {}

    for i in range(num_sensors):
        sensor_id = f"sensor_{i+1:02d}"

        # Generate base signal (seasonal + trend + noise)
        t = np.linspace(0, 4*np.pi, num_samples)
        base_signal = (2 * np.sin(t) + 0.5 * np.sin(5*t) +
                      0.1 * t + np.random.normal(0, 0.2, num_samples))

        # Add anomalies
        anomaly_mask = np.random.random(num_samples) < anomaly_prob
        base_signal[anomaly_mask] += np.random.normal(0, 3, np.sum(anomaly_mask))

        sensors_data[sensor_id] = base_signal

    return sensors_data


# Example usage and multi-sensor loop
def main():
    """
    Example demonstration of multi-sensor agent framework.
    Shows how to deploy multiple independent sensor agents.
    """
    print("=== IoT Sensor Agent Framework Demo ===\n")

    # Simulate sensor data
    print("1. Generating simulated sensor data...")
    sensors_data = simulate_sensor_data(num_sensors=12, num_samples=1500)
    window_length = 50

    # Initialize sensor agents
    print("2. Initializing sensor agents...")
    agents = {}
    for sensor_id in sensors_data.keys():
        agent = SensorAgent(
            sensor_id=sensor_id,
            model_type='lstm_autoencoder',  # Can vary per sensor
            window_length=window_length,
            memory_size=500,
            threshold_k=2.5,
            drift_threshold=0.15
        )
        agents[sensor_id] = agent

    # Train agents
    print("3. Training agents on historical data...")
    for sensor_id, agent in agents.items():
        # Use first 70% for training
        training_data = sensors_data[sensor_id][:int(0.7 * len(sensors_data[sensor_id]))]
        agent.fit(training_data)

    # Real-time detection simulation
    print("4. Running real-time anomaly detection...")
    print("-" * 80)

    # Process remaining data as streaming
    test_start = int(0.7 * len(list(sensors_data.values())[0]))
    anomaly_results = []

    for time_step in range(test_start, len(list(sensors_data.values())[0]) - window_length + 1):
        timestamp = datetime.now() + timedelta(seconds=time_step)
        step_anomalies = []

        # Process each sensor independently (parallel in real deployment)
        for sensor_id, agent in agents.items():
            if agent.is_fitted:
                # Extract current window
                subsequence = sensors_data[sensor_id][time_step:time_step + window_length]

                # Run detection
                result = agent.detect(subsequence, timestamp)

                # Collect anomalies for master agent escalation
                if result['is_anomaly'] or result['drift_flag']:
                    step_anomalies.append(result)

        # Report significant findings
        if step_anomalies:
            print(f"Time {time_step:4d}: {len(step_anomalies)} agents detected issues")
            for result in step_anomalies:
                status = "ANOMALY" if result['is_anomaly'] else ""
                status += " DRIFT" if result['drift_flag'] else ""
                print(f"  {result['sensor_id']}: {status} (score: {result['anomaly_score']:.3f}, "
                      f"conf: {result['confidence']:.2f})")

        anomaly_results.extend(step_anomalies)

    # Final status report
    print("\n" + "=" * 80)
    print("5. Final Agent Status Report:")
    print("-" * 80)

    for sensor_id, agent in agents.items():
        status = agent.get_status()
        print(f"{sensor_id}:")
        print(f"  Processed: {status['total_processed']} | "
              f"Anomalies: {status['anomalies_detected']} ({status['anomaly_rate']:.1%}) | "
              f"Drift Events: {status['drift_detected_count']} | "
              f"Memory: {status['memory_utilization']:.1%} | "
              f"Retrain: {'Yes' if status['should_retrain'] else 'No'}")

    print(f"\nTotal anomalies across all sensors: {len(anomaly_results)}")
    print("\n=== Demo Complete ===")

    return agents, anomaly_results


if __name__ == "__main__":
    # Run the demonstration
    agents, results = main()

    # Example of accessing individual agent for further analysis
    print("\nExample: Detailed status of first agent:")
    first_agent = list(agents.values())[0]
    detailed_status = first_agent.get_status()
    for key, value in detailed_status.items():
        print(f"  {key}: {value}")

error
RMSE: 1.3503097119121505 1.5002465652043568 2.837338449071969 1.5963914298880122 1.020955371098352
MAPE: 0.6891745407226914 0.7660456714610211 1.7857976818782204 0.750466542640601 0.377035791594537
MAE: 0.1685213302116819 0.18072839798571852 0.20651606722068036 0.1586292978244769 0.13245363757094158


In [ ]:
from google.colab import drive
drive.mount('/content/drive')